<a href="https://colab.research.google.com/github/MOEINSOHA/MOEINSOHA/blob/main/cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [11]:
import os
import shutil
# this code for remove some files
shutil.rmtree('/content/train/Colon_Benign/.ipynb_checkpoints')

In [11]:
!kaggle datasets download -d johncapocyan/cellnet-beta-version

Dataset URL: https://www.kaggle.com/datasets/johncapocyan/cellnet-beta-version
License(s): MIT
... resuming from 2513436672 bytes (7875837445 bytes left) ...
100% 9.66G/9.68G [01:31<00:00, 138MB/s]
100% 9.68G/9.68G [01:31<00:00, 85.6MB/s]


In [ ]:
!unzip /content/cellnet-beta-version.zip

In [15]:
import os
import shutil
images_path = '/content/CellNet/CellNet/Colon_Benign'
# name of images
all_items = os.listdir(images_path)
train_path = '/content/train/Colon_Benign'
# transfer image to file
for item in all_items:
    item_path = os.path.join(images_path, item)
    # searching for this the file is exsisted
    if os.path.isfile(item_path):
        shutil.move(item_path, train_path)
print(f"the files transferd")

the files transferd


In [16]:
images_path = '/content/CellNet/CellNet/Colon_Adenocarcinoma'
all_items = os.listdir(images_path)
train_path = '/content/train/Colon_Adenocarcinoma'
for item in all_items:
    item_path = os.path.join(images_path, item)
    if os.path.isfile(item_path):
        shutil.move(item_path, train_path)
print(f"the files transferd")

the files transferd


In [ ]:
import os
import random
import shutil


folder1 = "/content/train/Colon_Benign"
folder2 = "/content/train/Colon_Adenocarcinoma"


valid_folder1 = "/content/valid/Colon_Benign"
valid_folder2 = "/content/valid/Colon_Adenocarcinoma"
test_folder = "/content/test"

os.makedirs(valid_folder1, exist_ok=True)
os.makedirs(valid_folder2, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

def move_random_files(source_folder, destination_folder, percent=10):

    files = os.listdir(source_folder)
    random.shuffle(files)


    num_files_to_move = int(len(files) * percent / 100)


    for file in files[:num_files_to_move]:
        src_path = os.path.join(source_folder, file)
        dest_path = os.path.join(destination_folder, file)


        shutil.move(src_path, dest_path)
        print(f": {file} ")


move_random_files(folder1, valid_folder1, percent=20)
move_random_files(folder2, valid_folder2, percent=20)
move_random_files(folder1, test_folder, percent=5)
move_random_files(folder2, test_folder, percent=5)
print('ok')


In [18]:
fl_path = '/content/train/Colon_Adenocarcinoma'
contents = os.listdir(fl_path)
count = len(contents)
print(count)

3800


In [2]:
import os
# Define paths for training and testing datasets
train_dir = '/content/train'
valid_dir = '/content/valid'

# List the classes in the training directory
train_classes = os.listdir(train_dir)
print(f"Training classes: {train_classes}")

# Count the number of images in each class folder
for class_name in train_classes:
    class_path = os.path.join(train_dir, class_name)
    num_images = len(os.listdir(class_path))
    print(f"Number of images in {class_name}: {num_images}")


Training classes: ['Colon_Benign', 'Colon_Adenocarcinoma']
Number of images in Colon_Benign: 3799
Number of images in Colon_Adenocarcinoma: 3800


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
image_size = (224, 224)  # Image size expected by the model
batch_size = 32          # Number of images to process at once

# Create an ImageDataGenerator for training and validing
train_datagen = ImageDataGenerator(rescale=1.0/255)  # Only normalize pixel values
valid_datagen = ImageDataGenerator(rescale=1.0/255)   # Only normalize for valid data

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'  # Use (Colon_Adenocarcinoma and Colon_Benign)
)

# Load valid data
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'  #  two classes
)


Found 7599 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
print(train_generator.class_indices)  # Check the class indices for training data
print(valid_generator.class_indices)  # Check the class indices for validation data


{'Colon_Adenocarcinoma': 0, 'Colon_Benign': 1}
{'Colon_Adenocarcinoma': 0, 'Colon_Benign': 1}


In [5]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

# Create the ResNet model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add global average pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

# Add a fully connected layer for 2 classes
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Compile the model with categorical crossentropy for two classes
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=1,
    steps_per_epoch=len(train_generator),
    validation_steps=len(valid_generator)
)


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 91/238 ━━━━━━━━━━━━━━━━━━━━ 57:27 23s/step - accuracy: 0.9480 - loss: 0.1652